<a href="https://colab.research.google.com/github/Josafasouza/APP-GOODAY-FIGMAUXUI/blob/main/M5_Cardputer_como_Ponto_de_Acesso_Wi_Fi_(Corrigido).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import network
import socket
import time
import _thread

# Configurações da rede Wi-Fi que o Cardputer irá criar
AP_SSID = "M5CardputerAP"
AP_PASSWORD = "sua_senha"  # Troque por uma senha segura
AP_IP = "192.168.4.1"
AP_SUBNET = "255.255.255.0"
AP_GATEWAY = "192.168.4.1" # Gateway padrão para o Access Point

# Lista para armazenar informações dos clientes conectados
connected_clients = []

# Função para lidar com cada cliente conectado em uma thread separada
def handle_client(client_socket, client_address):
    print(f"Cliente conectado: {client_address}")
    connected_clients.append({"ip": client_address[0], "socket": client_socket})
    try:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break  # Cliente se desconectou
            print(f"Dados recebidos de {client_address}: {data.decode('utf-8')}")
            response = "Mensagem recebida do M5Cardputer!"
            client_socket.send(response.encode('utf-8'))
    except Exception as e:
        print(f"Erro na comunicação com {client_address}: {e}")
    finally:
        client_socket.close()
        print(f"Conexão com {client_address} encerrada.")
        # Remove o cliente da lista usando o socket para garantir a remoção correta
        for client in connected_clients:
            if client["socket"] == client_socket:
                connected_clients.remove(client)
                break

# Função para iniciar o ponto de acesso Wi-Fi
def start_access_point():
    try:
        ap = network.WLAN(network.AP_IF)
        ap.active(True)
        ap.config(essid=AP_SSID, password=AP_PASSWORD, authmode=network.AUTH_WPA_WPA2_PSK,
                  ip=AP_IP, subnet=AP_SUBNET, gateway=AP_GATEWAY) # Configura o Gateway
        print(f"Ponto de acesso Wi-Fi criado com SSID: {AP_SSID}, IP: {AP_IP}")

        # Cria um socket para ouvir conexões de clientes
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind(('0.0.0.0', 80))  # Usa a porta 80 para comunicação HTTP (exemplo)
        server_socket.listen(5)  # Permite até 5 conexões simultâneas na fila
        print("Aguardando conexões de clientes...")

        # Loop principal para aceitar novas conexões de clientes
        while True:
            client_socket, client_address = server_socket.accept()
            # Inicia uma nova thread para lidar com a comunicação com o cliente
            _thread.start_new_thread(handle_client, (client_socket, client_address))
    except Exception as e:
        print(f"Erro ao iniciar o ponto de acesso: {e}")
        print("Verifique se o MicroPython está instalado corretamente e se a placa suporta o modo AP.")
        print("O programa será encerrado.")
        return  # Encerra a função em caso de erro

# Função principal do script
if __name__ == "__main__":
    start_access_point() # Inicia o ponto de acesso
    while True:
        time.sleep(5)
        # Imprime informações sobre os clientes conectados a cada 5 segundos
        print("Clientes Conectados:")
        if not connected_clients:
            print("Nenhum cliente conectado.")
        else:
            for client in connected_clients:
                print(f"  IP: {client['ip']}")

ModuleNotFoundError: No module named 'network'